# 🎭 Three Character Text Generators
## Mickey Mouse 🐭 | Yoda 🟢 | Spider-Man 🕷️

This notebook fine-tunes 3 separate LoRA adapters for each character and creates an interactive chat interface.

**Characters:**
- 🐭 **Mickey Mouse** - Cheerful and optimistic mouse from Toontown
- 🟢 **Yoda** - Wise Jedi Master with unique backward speech pattern
- 🕷️ **Spider-Man** - Friendly neighborhood web-slinging superhero

---
## 🚀 Google Colab Setup (FREE GPU!)

**Follow these steps to run this notebook on Google Colab with free GPU:**

1. **Upload to Google Drive:**
   - Go to [Google Drive](https://drive.google.com)
   - Upload this notebook: `CharacterTextGenerators.ipynb`
   - Upload the 3 training data files: `mickeymouse.txt`, `yoda.txt`, `spiderman.txt`
   - Keep all files in the same folder

2. **Open in Colab:**
   - Right-click `CharacterTextGenerators.ipynb` in Google Drive
   - Select "Open with" → "Google Colaboratory"

3. **Enable GPU:**
   - In Colab, click "Runtime" → "Change runtime type"
   - Set "Hardware accelerator" to "T4 GPU" (free tier)
   - Click "Save"

4. **Run the notebook:**
   - The GPU check cell will confirm GPU is available
   - Training will take ~10-15 minutes (instead of hours on CPU!)

**Colab will automatically handle all library installations - no setup needed!**

---

---
## 1️⃣ Setup: Install Libraries and Load Training Data

In [ ]:
# Install required libraries
%pip install transformers peft datasets accelerate ipywidgets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from datasets import Dataset

print("✅ Libraries imported successfully!")

In [ ]:
# Check if GPU is available
import torch

print("🔍 Checking hardware availability...\n")

if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   Current Device: cuda:{torch.cuda.current_device()}")
    print("\n🎮 Training will use GPU acceleration!")
else:
    print("⚠️ WARNING: No GPU detected!")
    print("   Training will use CPU (much slower)")
    print("   Consider using Google Colab or a system with GPU")

print(f"\n📊 PyTorch Version: {torch.__version__}")

In [ ]:
# Load the character training data from text files
exec(open('mickeymouse.txt').read())
exec(open('yoda.txt').read())
exec(open('spiderman.txt').read())

print(f"✅ Mickey Mouse: {len(train_data_mickey)} training examples")
print(f"✅ Yoda: {len(train_data_yoda)} training examples")
print(f"✅ Spider-Man: {len(train_data_spider_man)} training examples")

---
## 2️⃣ Load Base Model and Tokenizer

In [ ]:
# Choose a small, efficient model
model_name = "Qwen/Qwen2-0.5B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"🔹 Base model loaded: {model_name}")
print(f"🔹 Parameters: {base_model.num_parameters():,}")

---
## 3️⃣ Training Function

In [ ]:
def train_character(character_name, train_data, output_dir, epochs=3):
    """
    Fine-tune a LoRA adapter for a specific character
    
    Args:
        character_name: Name of the character
        train_data: List of training examples
        output_dir: Directory to save the adapter
        epochs: Number of training epochs
    """
    print(f"\n{'='*70}")
    print(f"🎭 TRAINING {character_name.upper()}")
    print(f"{'='*70}\n")
    
    # Reload base model fresh for each character
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    
    # Configure LoRA
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )
    
    # Apply LoRA
    model = get_peft_model(model, lora_config)
    print(f"✅ LoRA adapter applied")
    model.print_trainable_parameters()
    
    # Format dataset using chat template
    def format_chat(example):
        text = tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False
        )
        return {"text": text}
    
    dataset = Dataset.from_list(train_data)
    dataset = dataset.map(format_chat)
    
    # Tokenize
    def tokenize_function(examples):
        result = tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=256
        )
        result["labels"] = result["input_ids"].copy()
        return result
    
    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text", "messages"])
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        learning_rate=2e-4,
        logging_steps=5,  # Log more frequently for better progress visibility
        save_strategy="epoch",
        report_to="none",
        fp16=True,
        use_cpu=False,  # Force GPU usage
        no_cuda=False,  # Enable CUDA
        disable_tqdm=False,  # Enable progress bars
        logging_first_step=True,  # Show initial progress
    )
    
    # Train
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
    )
    
    print(f"\n🔥 Starting training for {character_name}...\n")
    trainer.train()
    
    # Save
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    print(f"\n✅ {character_name} adapter saved to {output_dir}\n")
    
    # Clear memory
    del model
    del trainer
    torch.cuda.empty_cache()
    
    return output_dir

print("✅ Training function defined!")

---
## 4️⃣ Train All Three Characters

⚠️ **Note:** This will take approximately 10-15 minutes on a GPU (T4 or better recommended)

In [ ]:
# Train Mickey Mouse
# Note: Press the stop button (■) in the notebook toolbar to interrupt training if needed

import signal
import sys

def signal_handler(sig, frame):
    print('\n⚠️ Training interrupted by user!')
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

print("🐭 Training Mickey Mouse...")
print("💡 Tip: Click the stop button (■) to interrupt training\n")

mickey_adapter = train_character(
    character_name="Mickey Mouse",
    train_data=train_data_mickey,
    output_dir="./mickey-lora-adapter",
    epochs=3
)

In [ ]:
# Train Yoda
# Note: Press the stop button (■) in the notebook toolbar to interrupt training if needed

import signal
import sys

def signal_handler(sig, frame):
    print('\n⚠️ Training interrupted by user!')
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

print("🟢 Training Yoda...")
print("💡 Tip: Click the stop button (■) to interrupt training\n")

yoda_adapter = train_character(
    character_name="Yoda",
    train_data=train_data_yoda,
    output_dir="./yoda-lora-adapter",
    epochs=3
)

In [ ]:
# Train Spider-Man
# Note: Press the stop button (■) in the notebook toolbar to interrupt training if needed

import signal
import sys

def signal_handler(sig, frame):
    print('\n⚠️ Training interrupted by user!')
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

print("🕷️ Training Spider-Man...")
print("💡 Tip: Click the stop button (■) to interrupt training\n")

spiderman_adapter = train_character(
    character_name="Spider-Man",
    train_data=train_data_spider_man,
    output_dir="./spiderman-lora-adapter",
    epochs=3
)

print("\n" + "="*70)
print("🎉 ALL THREE CHARACTERS TRAINED SUCCESSFULLY!")
print("="*70)

---
## 5️⃣ Character Chat System

Create an interactive chat interface to talk with any of the three characters!

In [ ]:
class CharacterChat:
    """Interactive chat with multiple character personalities"""
    
    def __init__(self):
        self.characters = {
            "Mickey Mouse": {
                "adapter": "./mickey-lora-adapter",
                "emoji": "🐭",
                "description": "Cheerful and optimistic mouse from Toontown"
            },
            "Yoda": {
                "adapter": "./yoda-lora-adapter",
                "emoji": "🟢",
                "description": "Wise Jedi Master who speaks in unique way"
            },
            "Spider-Man": {
                "adapter": "./spiderman-lora-adapter",
                "emoji": "🕷️",
                "description": "Friendly neighborhood web-slinging hero"
            }
        }
        
        self.current_character = None
        self.current_model = None
        self.base_model = None
        self.tokenizer = None
        
    def load_character(self, character_name):
        """Load a specific character's LoRA adapter"""
        if character_name not in self.characters:
            print(f"❌ Character '{character_name}' not found!")
            return False
        
        print(f"\n🔄 Loading {character_name}...")
        
        # Clear previous model if exists
        if self.current_model is not None:
            del self.current_model
            torch.cuda.empty_cache()
        
        # Load base model if not loaded
        if self.base_model is None:
            self.base_model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
        
        # Load tokenizer if not loaded
        if self.tokenizer is None:
            adapter_path = self.characters[character_name]["adapter"]
            self.tokenizer = AutoTokenizer.from_pretrained(adapter_path)
        
        # Load character's LoRA adapter
        adapter_path = self.characters[character_name]["adapter"]
        self.current_model = PeftModel.from_pretrained(self.base_model, adapter_path)
        self.current_model.eval()
        
        self.current_character = character_name
        
        emoji = self.characters[character_name]["emoji"]
        print(f"✅ {emoji} {character_name} loaded and ready to chat!\n")
        return True
    
    def chat(self, user_message, max_tokens=50, temperature=0.7):
        """Generate a response from the current character"""
        if self.current_model is None:
            return "❌ No character loaded! Please load a character first."
        
        # Format message using chat template
        messages = [{"role": "user", "content": user_message}]
        prompt = self.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        # Generate response
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.current_model.device)
        
        with torch.no_grad():
            outputs = self.current_model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=True,
                temperature=temperature,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        # Decode response
        response = self.tokenizer.decode(
            outputs[0][len(inputs['input_ids'][0]):], 
            skip_special_tokens=True
        )
        
        return response
    
    def show_characters(self):
        """Display available characters"""
        print("\n" + "="*70)
        print("🎭 AVAILABLE CHARACTERS:")
        print("="*70)
        for i, (name, info) in enumerate(self.characters.items(), 1):
            emoji = info["emoji"]
            desc = info["description"]
            current = " ⭐ (CURRENT)" if name == self.current_character else ""
            print(f"\n{i}. {emoji} {name}{current}")
            print(f"   {desc}")
        print("\n" + "="*70 + "\n")

# Create the chat instance
chat = CharacterChat()
print("✅ Character Chat System initialized!")

---
## 6️⃣ View Available Characters

In [ ]:
chat.show_characters()

---
## 7️⃣ Chat with Mickey Mouse 🐭

In [ ]:
chat.load_character("Mickey Mouse")

# Test with different prompts
test_prompts = [
    "Hello! How are you today?",
    "What's your favorite thing to do?",
    "Tell me about your friends",
]

for prompt in test_prompts:
    response = chat.chat(prompt)
    print(f"👤 You: {prompt}")
    print(f"🐭 Mickey: {response}")
    print("-" * 70 + "\n")

---
## 8️⃣ Chat with Yoda 🟢

In [ ]:
chat.load_character("Yoda")

test_prompts = [
    "Hello! How are you today?",
    "What is the Force?",
    "Can you teach me to be a Jedi?",
]

for prompt in test_prompts:
    response = chat.chat(prompt)
    print(f"👤 You: {prompt}")
    print(f"🟢 Yoda: {response}")
    print("-" * 70 + "\n")

---
## 9️⃣ Chat with Spider-Man 🕷️

In [ ]:
chat.load_character("Spider-Man")

test_prompts = [
    "Hello! How are you today?",
    "How do you swing between buildings?",
    "What's it like being a hero?",
]

for prompt in test_prompts:
    response = chat.chat(prompt)
    print(f"👤 You: {prompt}")
    print(f"🕷️ Spider-Man: {response}")
    print("-" * 70 + "\n")

---
## 🔟 Interactive Chat Interface

Chat freely with any character! Switch between them anytime.

In [ ]:
def interactive_chat():
    """
    Interactive chat interface - chat with any character!
    
    Commands:
    - Type your message to chat with the current character
    - Type 'switch' to change character
    - Type 'list' to see available characters
    - Type 'quit' to exit
    """
    
    print("\n" + "="*70)
    print("🎭 INTERACTIVE CHARACTER CHAT")
    print("="*70)
    print("\nCommands:")
    print("  - Type your message to chat with the current character")
    print("  - Type 'switch' to change character")
    print("  - Type 'list' to see available characters")
    print("  - Type 'quit' to exit")
    print("\n" + "="*70 + "\n")
    
    # Load first character if none loaded
    if chat.current_character is None:
        chat.load_character("Mickey Mouse")
    
    while True:
        # Get current character info
        emoji = chat.characters[chat.current_character]["emoji"]
        
        # Get user input
        user_input = input(f"\n👤 You: ").strip()
        
        if not user_input:
            continue
        
        # Handle commands
        if user_input.lower() == 'quit':
            print("\n👋 Goodbye! Thanks for chatting!\n")
            break
        
        elif user_input.lower() == 'list':
            chat.show_characters()
            continue
        
        elif user_input.lower() == 'switch':
            chat.show_characters()
            
            character_choice = input("Enter character name (or press Enter to cancel): ").strip()
            
            if character_choice and character_choice in chat.characters:
                chat.load_character(character_choice)
            elif character_choice:
                print(f"❌ '{character_choice}' not found. Staying with {chat.current_character}")
            
            continue
        
        # Generate response
        response = chat.chat(user_input)
        print(f"{emoji} {chat.current_character}: {response}")

# Ready to start
print("💬 Ready to start interactive chat!")
print("Run: interactive_chat()")

In [ ]:
# Uncomment and run to start chatting!
# interactive_chat()

---
## 🎪 BONUS: Character Comparison

Ask the same question to all three characters and compare their responses!

In [ ]:
def compare_characters(question):
    """Ask the same question to all three characters"""
    
    print("\n" + "="*70)
    print(f"❓ QUESTION: {question}")
    print("="*70 + "\n")
    
    for character_name in ["Mickey Mouse", "Yoda", "Spider-Man"]:
        chat.load_character(character_name)
        response = chat.chat(question, max_tokens=60)
        
        emoji = chat.characters[character_name]["emoji"]
        print(f"{emoji} {character_name}:")
        print(f"   {response}\n")
        print("-" * 70 + "\n")

print("✅ Comparison function ready!")

In [ ]:
# Test with various questions
questions = [
    "What makes you happy?",
    "What's your biggest challenge?",
    "Do you have any advice for me?",
]

for q in questions:
    compare_characters(q)

---
## 🎯 Summary

You've successfully created 3 character text generators!

**What you can do:**
1. ✅ Chat with Mickey Mouse, Yoda, or Spider-Man
2. ✅ Switch between characters anytime
3. ✅ Compare how each character responds to the same question
4. ✅ Use the interactive chat for natural conversations

**Each character maintains their unique personality:**
- 🐭 Mickey Mouse: Cheerful, optimistic, uses "Oh boy!" and "Haha!"
- 🟢 Yoda: Wise, speaks in reverse syntax, uses "Hmm" and "Yes"
- 🕷️ Spider-Man: Witty, heroic, makes jokes about swinging and web-slinging

**Try your own prompts and have fun chatting!** 🎉